### 客户逾期还款业务

- 数据源结构：数据集是Lending Club平台发生借贷的业务数据，共有52个变量，39522条记录。


<img src="图片/数据集2.png" width = "800" height = "800" div align=centre />

<img src="图片/数据源结构.png" width = "400" height = "400" div align=centre />

#### 数据预处理

In [15]:
import numpy as np
import warnings
warnings.filterwarnings('ignore')

- 去掉一些明显没用的特征，如’desc’,'url’，并将剩下特征保存到一个新的csv文件中

In [2]:
import pandas as pd

loans_2020 = pd.read_csv('LoanStats3a.csv', skiprows=1) #第一行是字符串
half_count = len(loans_2020) / 2

loans_2020 = loans_2020.dropna(thresh=half_count, axis=1) #剔除空白值超过一半的列
loans_2020 = loans_2020.drop(['desc', 'url'], axis=1) #按照列中，删除描述和URL链接
loans_2020.to_csv('loans_2020.csv', index=False) #index=False表示不加索引

- 输出数据标签，初判断无用特征

In [5]:
pd.set_option('display.max_columns', None)

loans_2020 = pd.read_csv("loans_2020.csv")
#print("第一行的数据展示 \n",loans_2020.iloc[0])  #第一行的数据

In [6]:
loans_2020.dtypes

id                             object
member_id                     float64
loan_amnt                     float64
funded_amnt                   float64
funded_amnt_inv               float64
term                           object
int_rate                       object
installment                   float64
grade                          object
sub_grade                      object
emp_title                      object
emp_length                     object
home_ownership                 object
annual_inc                    float64
verification_status            object
issue_d                        object
loan_status                    object
pymnt_plan                     object
purpose                        object
title                          object
zip_code                       object
addr_state                     object
dti                           float64
delinq_2yrs                   float64
earliest_cr_line               object
inq_last_6mths                float64
open_acc    

In [7]:
print("原始列数= ",loans_2020.shape[1])

原始列数=  52


- 很明显从常识来讲‘id’和‘member_id’与银行是否对他进行放贷没有任何关系，’funded_amnt(期望贷款的值)’ 和funded_amnt_inv(实际发放的款数)’为预测之后银行对该人借贷的金额，很明显也与模型判断没有关系。

In [8]:
#id：用户ID
#member_id：会员编号
#funded_amnt：承诺给该贷款的总金额
#funded_amnt_inv：投资者为该贷款承诺的总金额
#grade：贷款等级。贷款利率越高，则等级越高
#sub_grade：贷款子等级
#emp_title：工作名称
#issue_d：贷款月份

loans_2020 = loans_2020.drop(["id", "member_id", "funded_amnt", "funded_amnt_inv", "grade", "sub_grade", "emp_title", "issue_d"], axis=1)

In [9]:
#loan_status：Fully Paid——>全部付讫  Charged Off——>没有按时还款  


loans_2020['loan_status'].value_counts()

Fully Paid            33693
Charged Off            5612
Current                 201
Late (31-120 days)       10
In Grace Period           9
Late (16-30 days)         5
Default                   1
Name: loan_status, dtype: int64

In [10]:
#zip_code：常用的邮编
#out_prncp和out_prncp_inv都是一样的：总资金中剩余的未偿还本金
#out_prncp_inv：实际未偿还的本金
#total_rec_prncp：迄今收到的本金



loans_2020 = loans_2020.drop(["zip_code", "out_prncp", "out_prncp_inv", "total_pymnt", "total_pymnt_inv", "total_rec_prncp"], axis=1)

In [11]:
#total_rec_int：迄今收到的利息

#recoveries：是否收回本金
#collection_recovery_fee：收集回收费用
#last_pymnt_d：最近一次收到还款的时间
#last_pymnt_amnt：全部的还款的时间


#保留候选特征
loans_2020 = loans_2020.drop(["total_rec_int", "total_rec_late_fee", "recoveries", "collection_recovery_fee", "last_pymnt_d", "last_pymnt_amnt"], axis=1)
loans_2020.dtypes

loan_amnt                     float64
term                           object
int_rate                       object
installment                   float64
emp_length                     object
home_ownership                 object
annual_inc                    float64
verification_status            object
loan_status                    object
pymnt_plan                     object
purpose                        object
title                          object
addr_state                     object
dti                           float64
delinq_2yrs                   float64
earliest_cr_line               object
inq_last_6mths                float64
open_acc                      float64
pub_rec                       float64
revol_bal                     float64
revol_util                     object
total_acc                     float64
initial_list_status            object
last_credit_pull_d             object
collections_12_mths_ex_med    float64
policy_code                   float64
application_

In [12]:
print("现在的列数 = ",loans_2020.shape[1]) #原始是52列，现在是32列的候选特征

现在的列数 =  32


- 在原始数据中,没有0或者1的还不还款的特征，但是有“loan status”这个特征，意思是当前“贷款的状态”

- 确定当前贷款状态（label值）

In [13]:
loans_2020['loan_status'].value_counts()


#Fully Paid：批准了客户的贷款，后面给他打个“1”
#Charged Off：没有批准了客户的贷款，后面给他打个“0”
#Does not meet the credit policy. Status:Fully Paid：，没有满足要求的有1988个，也不要说清楚不贷款，就不要这个属性了
#后面的属性不确定比较强
#Late (16-30 days)  ：延期了16-30 days
#Late (31-120 days)：延期了31-120 days

Fully Paid            33693
Charged Off            5612
Current                 201
Late (31-120 days)       10
In Grace Period           9
Late (16-30 days)         5
Default                   1
Name: loan_status, dtype: int64

In [14]:
loans_2020['loan_status'].head(10)

0     Fully Paid
1    Charged Off
2     Fully Paid
3     Fully Paid
4        Current
5     Fully Paid
6     Fully Paid
7     Fully Paid
8    Charged Off
9    Charged Off
Name: loan_status, dtype: object

- 要做一个二分类label，用以下两种方法

loans_2020 = loans_2020[(loans_2020['loan_status'] == "Fully Paid") |
                        (loans_2020['loan_status'] == "Charged Off")]
status_replace = {
    "loan_status": {
        "Fully Paid": 1, #完全支付
        "Charged Off": 0,#违约
    }
}

loans_2020 = loans_2020.replace(status_replace)

In [16]:
loans_2020 = loans_2020[loans_2020['loan_status'].isin(["Fully Paid", "Charged Off"])]

loans_2020['loan_status'] = np.where(loans_2020['loan_status']=="Fully Paid", 1, 0)

In [17]:
loans_2020['loan_status'].value_counts()

1    33693
0     5612
Name: loan_status, dtype: int64

- 去掉特征中只有一种属性的列

In [18]:
#某列特征都是n n n  NaN  n n ,有缺失的，唯一的属性就有2个，用pandas空值给去掉

orig_columns = loans_2020.columns  #展现出所有的列

drop_columns = []  #初始化空值

for col in orig_columns:
    col_series = loans_2020[col].dropna().unique()  #去重唯一的属性
    if len(col_series) == 1:  #如果该特征的属性只有一个属性，就给过滤掉该特征
        drop_columns.append(col)
        
loans_2020 = loans_2020.drop(drop_columns, axis=1)
print(drop_columns)
print("--------------------------------------------")
print(loans_2020.shape)

loans_2020.to_csv('filtered_loans_2020.csv', index=False)

['initial_list_status', 'collections_12_mths_ex_med', 'policy_code', 'application_type', 'acc_now_delinq', 'chargeoff_within_12_mths', 'delinq_amnt', 'tax_liens']
--------------------------------------------
(39305, 24)


- 24个列的缺失值情况，统计一下每列的缺失值

- 处理缺失值

In [19]:
loans = pd.read_csv('filtered_loans_2020.csv')
null_counts = loans.isnull().sum()  #用pandas的isnull统计一下每列的缺失值，给累加起来
print(null_counts) 

#对于每列中缺失的情况不是很大，大多数是很好的 ，那就删掉几个列也无可厚非(对于样本大)，或者是只删除缺失值，或者用均值、中位数和众数补充

loan_amnt                  0
term                       0
int_rate                   0
installment                0
emp_length              1073
home_ownership             0
annual_inc                 0
verification_status        0
loan_status                0
pymnt_plan                 0
purpose                    0
title                     11
addr_state                 0
dti                        0
delinq_2yrs                0
earliest_cr_line           0
inq_last_6mths             0
open_acc                   0
pub_rec                    0
revol_bal                  0
revol_util                50
total_acc                  0
last_credit_pull_d         1
pub_rec_bankruptcies     449
dtype: int64


- 从统计出的结果可以看出‘title’和‘revol_util’相对于数据总量来说较少，可以直接去掉缺失值所在的行，而‘pub_rec_bankruptcies ’中的缺失值较多，说明该数据统计的情况较差，在本文中直接将此特征删除即可。
emp_length中的空值稍后以0值处理

In [20]:
loans = loans.drop("pub_rec_bankruptcies", axis=1)
loans = loans.dropna(axis=0) 

print(loans.dtypes.value_counts())

object     12
float64    10
int64       1
dtype: int64


- 数据类型的转换

    由于sk-learn库不接受字符型的数据，所以还需将上面特征中12个字符型的数据进行处理。

In [21]:
object_columns_df = loans.select_dtypes(include=["object"])
print(object_columns_df.iloc[0])


#term：分期多少个月啊
#int_rate：利息，10.65%，后面还要把%去掉
#emp_length：10年的映射成10，9年的映射成9
#home_ownership：房屋所有权，是租的、还是自己的、还是低压出去了，那就用0 1 2来代替

term                     36 months
int_rate                    10.65%
emp_length               10+ years
home_ownership                RENT
verification_status       Verified
pymnt_plan                       n
purpose                credit_card
title                     Computer
addr_state                      AZ
earliest_cr_line            Jan-85
revol_util                  83.70%
last_credit_pull_d          Nov-16
Name: 0, dtype: object


In [22]:
#查看指定标签的属性，并记数
#home_ownership：房屋所有权
#verification_status：身份保持证明
#emp_length：客户公司名称
#term：贷款分期的时间
#addr_state：地址邮编


cols = [
    'home_ownership', 'verification_status', 'emp_length', 'term', 'addr_state'
]
for c in cols:
    print(loans[c].value_counts())
    print('-'*20)


RENT        18237
MORTGAGE    17035
OWN          2805
OTHER          96
NONE            1
Name: home_ownership, dtype: int64
--------------------
Not Verified       16182
Verified           12251
Source Verified     9741
Name: verification_status, dtype: int64
--------------------
10+ years    8794
< 1 year     4492
2 years      4339
3 years      4052
4 years      3397
5 years      3262
1 year       3182
6 years      2201
7 years      1747
8 years      1463
9 years      1245
Name: emp_length, dtype: int64
--------------------
 36 months    27980
 60 months    10194
Name: term, dtype: int64
--------------------
CA    6876
NY    3644
FL    2739
TX    2657
NJ    1799
IL    1478
PA    1470
VA    1355
GA    1342
MA    1278
OH    1176
MD    1019
AZ     824
WA     788
CO     758
NC     739
CT     725
MI     688
MO     654
MN     589
NV     477
SC     457
OR     431
WI     429
AL     424
LA     422
KY     320
OK     292
KS     257
UT     248
AR     233
DC     211
RI     196
NM     180
HI     1

In [23]:
#"purpose"和"title"表达的意思相近，且从输出结果可以看出"title"所含的属性较多，可以将其舍弃掉
print(loans["purpose"].value_counts())
#purpose：你贷款时的目的是什么，买房还是买车，还是其他消费

print("------------------------------------------------")

print(loans["title"].value_counts())
#title：跟purpose一样，贷款的目的，选一个就行了

debt_consolidation    18057
credit_card            4927
other                  3761
home_improvement       2846
major_purchase         2103
small_business         1745
car                    1489
wedding                 924
medical                 665
moving                  551
house                   364
vacation                347
educational             300
renewable_energy         95
Name: purpose, dtype: int64
------------------------------------------------
Debt Consolidation                       2122
Debt Consolidation Loan                  1670
Personal Loan                             625
Consolidation                             502
debt consolidation                        483
Credit Card Consolidation                 348
Home Improvement                          343
Debt consolidation                        320
Small Business Loan                       310
Credit Card Loan                          302
Personal                                  296
Consolidation Loan       

<img src="图片/分割线.png" width = "800" height = "400" div align=left />

In [24]:
mapping_dict = {
    "emp_length": {
        "10+ years": 10,
        "9 years": 9,
        "8 years": 8,
        "7 years": 7,
        "6 years": 6,
        "5 years": 5,
        "4 years": 4,
        "3 years": 3,
        "2 years": 2,
        "1 year": 1,
        "< 1 year": 0,
        "n/a": 0
    }
}

# 删除：last_credit_pull_d：LC撤回最近的月份   
#earliest_cr_line：第一次借贷时间
#addr_state：家庭邮编
#title：URL的标题
loans = loans.drop(
    ["last_credit_pull_d", "earliest_cr_line", "addr_state", "title"], axis=1)
#rstrip：删除 string 字符串末尾的指定字符
loans["int_rate"] = loans["int_rate"].str.rstrip("%").astype("float")
#revol_util：透支额度占信用比例
loans["revol_util"] = loans["revol_util"].str.rstrip("%").astype("float")
loans = loans.replace(mapping_dict)

- 剩余的其他字符型特征，此处选择使用pandas的get_dummies()函数，直接映射为数值型。

In [25]:
#查看指定标签的属性，并记数
#home_ownership：房屋所有权
#verification_status：身份保持证明
#emp_length：客户公司名称
#purpose：贷款的意图
#term：贷款分期的时间

cat_columns = ["home_ownership", "verification_status", "emp_length", "purpose", "term"]
dummy_df = pd.get_dummies(loans[cat_columns])

loans = pd.concat([loans, dummy_df], axis=1)
loans = loans.drop(cat_columns, axis=1)
#pymnt_plan 指示是否已为贷款实施付款计划 ，里面都为N，删掉这一列
loans = loans.drop("pymnt_plan", axis=1)

In [26]:
loans.to_csv('cleaned_loans_2020.csv', index=False)

In [27]:
loans = pd.read_csv("cleaned_loans_2020.csv") # 清洗完的数据拿过来，现在的数据要么是float类型和int类型
print(loans.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38174 entries, 0 to 38173
Data columns (total 48 columns):
loan_amnt                              38174 non-null float64
int_rate                               38174 non-null float64
installment                            38174 non-null float64
annual_inc                             38174 non-null float64
loan_status                            38174 non-null int64
dti                                    38174 non-null float64
delinq_2yrs                            38174 non-null float64
inq_last_6mths                         38174 non-null float64
open_acc                               38174 non-null float64
pub_rec                                38174 non-null float64
revol_bal                              38174 non-null float64
revol_util                             38174 non-null float64
total_acc                              38174 non-null float64
home_ownership_MORTGAGE                38174 non-null int64
home_ownership_NONE        

#### 模型训练

- 对于二分类问题，一般情况下，首选逻辑回归。
首先定义模型效果的评判标准。根据贷款行业的实际情况，在这里我们假设将钱借给了没有还款能力的人，结果损失一千，将钱借给了有偿还能力的人，从每笔中赚0.1的利润，而其余情况收益为零，就相当于预测对十个人才顶上预测错一个人的收益，所以精度不再适用于此模型，为了实现利润最大化，不仅要求模型预测recall率较高，同时是需要要让fall-out率较低，故这里采用两个指标TPR(true positive rate)和FPR(false positive rate)。

In [28]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression() # 调用逻辑回归的算法包



cols = loans.columns # 4万行 * 24列的样本
train_cols = cols.drop("loan_status") # 删除loan_status这一列作为目标值

features = loans[train_cols] # 23列的特征矩阵
target = loans["loan_status"] # 作为标签矩阵

In [31]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(features, target)

In [32]:
lr.fit(X_train, Y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [33]:
lr.score(X_test, Y_test)

0.856035205364627

In [34]:
predictions = lr.predict(X_test)

In [35]:
predictions[:10] #0:代表没有偿还  1:代表偿还

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [36]:
lr.predict_proba(X_test)#lr的概率模型

array([[0.10807275, 0.89192725],
       [0.17030142, 0.82969858],
       [0.11969455, 0.88030545],
       ...,
       [0.14419433, 0.85580567],
       [0.11863675, 0.88136325],
       [0.11777812, 0.88222188]])

- 咱们的目的是服务的金融俱乐部，让它挣个有能力偿还的客户的利息，怎么挣这个钱呢？


- 第一个实际值为0，客户不会还钱，模型预测客户能还钱，为1 ，假设系统贷给了客户1000块钱，但是一分都没还，说明预测错了为阴性，赔了1000块钱

- 第二个实际值为1，客户有偿还能力，模型预测客户有能力偿还，咱们就挣了客户的利息钱，1000*0.1 =100块钱

- 第三个实际值为0，本来客户是不还钱的，模型预测他也不还钱，咱们也没借他

- 第四个客户能还钱，模型预测客户不能还 ，咱们也没有贷款给他

#### 建立混淆矩阵

In [37]:
import pandas as pd
#接下来就是如何算4个指标 fp tp fn tn

# 假正类（False Positive，FP）：将负类预测为正类
fp_filter = (predictions == 1) & (Y_test == 0)
fp = len(predictions[fp_filter])
print(f'fp:{fp}')
print("----------------------------------------")


# 真正类（True Positive，TP）：将正类预测为正类
tp_filter = (predictions == 1) & (Y_test == 1)
tp = len(predictions[tp_filter])
print(f'tp:{tp}')
print("----------------------------------------")


# 假负类（False Negative，FN）：将正类预测为负类
fn_filter = (predictions == 0) & (Y_test == 1)
fn = len(predictions[fn_filter])
print(f'fn:{fn}')
print("----------------------------------------")

# 真负类（True Negative，TN）：将负类预测为负类
tn_filter = (predictions == 0) & (Y_test == 0)
tn = len(predictions[tn_filter])
print(f'tn:{tn}')

fp:1366
----------------------------------------
tp:8167
----------------------------------------
fn:8
----------------------------------------
tn:3


In [38]:
from sklearn.metrics import confusion_matrix, classification_report

In [39]:
confusion_matrix(Y_test, predictions)

array([[   3, 1366],
       [   8, 8167]])

In [41]:
print(classification_report(Y_test, predictions))

              precision    recall  f1-score   support

           0       0.27      0.00      0.00      1369
           1       0.86      1.00      0.92      8175

    accuracy                           0.86      9544
   macro avg       0.56      0.50      0.46      9544
weighted avg       0.77      0.86      0.79      9544



- 这个数据集后续观察是不平衡的 ，借钱的有6个，不借钱的有1个，借钱的样本本来就多，不借钱的样本本来就少，相当于7个人来了，有6个人借给他了为1，有1个人没借给他为0，7个样本的错误率为1/7 ,准确率为6/7

<img src="图片/盈亏情况.png" width = "400" height = "400" div align=centre />

- 第一个实际值为0，没有偿还能力，模型预测客户为1 ，代表能还，赔了1000块钱 

- 后面的实际值为1，代表有偿还能力，模型借给他1000块钱，挣了个利息钱100块
- 最终-1000 + 600 = - 400 用”精度“来衡量最终还是会赔钱的，因为数据集能还钱的样本很多，显然这样是不合理的，所以就不考虑”精度“了。

In [30]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict

lr = LogisticRegression()
predictions = cross_val_predict(lr, features, target, cv=10) # Kfold = 10(交叉验证)
predictions = pd.Series(predictions)
print(predictions[:1000])

0      1
1      1
2      1
3      1
4      1
      ..
995    1
996    1
997    1
998    1
999    1
Length: 1000, dtype: int64


In [31]:
# 假正类（False Positive，FP）：将负类预测为正类
fp_filter = (predictions == 1) & (loans["loan_status"] == 0)
fp = len(predictions[fp_filter])


# 真正类（True Positive，TP）：将正类预测为正类
tp_filter = (predictions == 1) & (loans["loan_status"] == 1)
tp = len(predictions[tp_filter])



# 假负类（False Negative，FN）：将正类预测为负类
fn_filter = (predictions == 0) & (loans["loan_status"] == 1)
fn = len(predictions[fn_filter])



# 真负类（True Negative，TN）：将负类预测为负类
tn_filter = (predictions == 0) & (loans["loan_status"] == 0)
tn = len(predictions[tn_filter])


- 真正率TPR是指客户的实际值为1，有偿还能力，模型预测也为1，说明这些客户群体越来越多，挣的利息也越来越多，所以啊，我们希望真正率TPR越高越好

- 10个客户，其中有3个是没有还款能力 ，7个数有还款能力的，银行进行了系统预测，结果有3个没有还款能力的人，其中有2个人确实没有还款能力，那么TPR(真正率) = 2/3 ，对于7位有按时还贷的人来说，有一位被误认为是没有能力还款，那边FPR(假正率) = 1/7

- 总结：本质上期望TPR越高越好，FPR越低越好

In [32]:
# Rates：就可以用刚才的指标进行衡量了呀
tpr = tp / float((tp + fn))
fpr = fp / float((fp + tn))
"""
tpr:比较高，我们非常喜欢，给他贷款了，而且这些人能还钱了
fpr：比较高，这些人不会还钱，但还是贷给他了吧
为什么这个2个值都那么高呢？把所有人来了，都借给他钱呀，打印出前20行都为1，为什么会出现这种情况？
绝对是前面的数据出现问题了，比如说数据是6：1，绝大多数是1，小部分是0，样本不均衡的情况下，导致分类器错误的认为把所有的样本预测为1，因为负样本少，咱们就“数据增强”，
把负样本1增强到4份儿，是不是可以啊，要么收集数据 ，数据已经定值了，没办法收集，要么是造数据，你知道什么样的人会还钱吗？也不好造吧，怎么解决样本不均衡的问题呢？
接下来要考虑权重的东西了，一部分是6份，另一部分是1份，把6份的权重设置为1，把1份的权重设置为6，设置权重项来进行衡量，把不均衡的样本变得均衡，加了权重项，让正样本对结果的影响小一些，
让负样本对结果的影响大一些，通过加入权重项，模型对结果变得均衡一下，有一个参数很重要
"""
print(tpr)#真正率
print(fpr)#假正率
print(predictions[:20])

0.9990856167515011
0.9983224603914259
0     1
1     1
2     1
3     1
4     1
5     1
6     1
7     1
8     1
9     1
10    1
11    1
12    1
13    1
14    1
15    1
16    1
17    1
18    1
19    1
dtype: int64


<img src="图片/分割线.png" width = "800" height = "400" div align=left />

- 从结果看出TPR和FPR的值都很高，说明来一个人基本都会判断为可以借钱，那显然模型就完全没有分类的意义。

#### 4.1.4 考虑权重后使用逻辑回归训练

In [33]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict
"""
class_weight：可以调整正反样本的权重
balanced:希望正负样本平衡一些的
"""
lr = LogisticRegression(class_weight="balanced")
predictions = cross_val_predict(lr, features, target, cv=10)
predictions = pd.Series(predictions)

# False positives.
fp_filter = (predictions == 1) & (loans["loan_status"] == 0)
fp = len(predictions[fp_filter])

# True positives.
tp_filter = (predictions == 1) & (loans["loan_status"] == 1)
tp = len(predictions[tp_filter])

# False negatives.
fn_filter = (predictions == 0) & (loans["loan_status"] == 1)
fn = len(predictions[fn_filter])

# True negatives
tn_filter = (predictions == 0) & (loans["loan_status"] == 0)
tn = len(predictions[tn_filter])

# Rates
tpr = tp / float((tp + fn))
fpr = fp / float((fp + tn))

print(tpr)#真正率
print()
print(fpr)#假正率
print()
print(predictions[:20])

0.6585692950105154

0.38341099720410066

0     1
1     0
2     0
3     0
4     1
5     0
6     0
7     0
8     0
9     0
10    1
11    0
12    1
13    1
14    0
15    0
16    1
17    1
18    1
19    0
dtype: int64


#### 4.1.5 自定义权重后使用逻辑回归训练

In [34]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict
"""
权重项可以自己定义的
0代表5倍的
1代表10倍的
"""
penalty = {
    0: 5,
    1: 1
}

lr = LogisticRegression(class_weight=penalty)
# kf = KFold(features.shape[0], random_state=1)
kf = 10
predictions = cross_val_predict(lr, features, target, cv=kf)
predictions = pd.Series(predictions)

# False positives.
fp_filter = (predictions == 1) & (loans["loan_status"] == 0)
fp = len(predictions[fp_filter])

# True positives.
tp_filter = (predictions == 1) & (loans["loan_status"] == 1)
tp = len(predictions[tp_filter])

# False negatives.
fn_filter = (predictions == 0) & (loans["loan_status"] == 1)
fn = len(predictions[fn_filter])

# True negatives
tn_filter = (predictions == 0) & (loans["loan_status"] == 0)
tn = len(predictions[tn_filter])

# Rates
tpr = tp / float((tp + fn))
fpr = fp / float((fp + tn))

print(tpr)
print()
print(fpr)

0.7343107074278399

0.4797763280521901


- 那么为什么会出现上面极其离谱的现象呢？这是由于我们的样本是很不均衡的，这就容易导致我们构建的分类器把所有样本都归为样本量较大的那一个类。解决的方法有很多，其中一个是进行数据增强，就是把少的样本增多，但是要添加的数据要么是收集的，要么是自己造的，所以这项工作还是挺难的。所以将考虑权重，将少的样本的权重增大，期望模型能够达到比较均衡的状态。

 - 对上述模型的预测结果进行简单的分析，发现错误率和正确率都达到99.9%，错误率太高，通过观察预测结果发现，模型几乎将所有的样本都判断为正例，通过对原始数据的了解，分析造成该现象的原因是由于政府样本数量相差太大，即样本不均衡造成模型对正例样本有所偏重，这里采用对样本添加权重值的方式进行调整，首先采用默认的均衡调整。


- 以上案例不是着重给出一个正确率的预测模型，只是给出使用机器学习建模的一般流程，分为两大部分：数据处理和模型学习，第一部分需要大量的知识对原始数据进行清理及特征提取，第二部分模型学习，涉及长时间的模型参数调整，调整方向和策略需要大家进一步的研究。模型效果不理想时，可以考虑的调整策略：

    1.调节正负样本的权重参数。

    2.更换模型算法。

    3.同时几个使用模型进行预测，然后取去测的最终结果。

    4.使用原数据，生成新特征。

    5.调整模型参数
